In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'], [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'], [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'], [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype({'user_id':'Int64', 'session_id':'Int64', 'activity_date':'datetime64[ns]', 'activity_type':'object'})

In [4]:
activity = spark.createDataFrame(activity)
activity.show()

+-------+----------+-------------------+-------------+
|user_id|session_id|      activity_date|activity_type|
+-------+----------+-------------------+-------------+
|      1|         1|2019-07-20 00:00:00| open_session|
|      1|         1|2019-07-20 00:00:00|  scroll_down|
|      1|         1|2019-07-20 00:00:00|  end_session|
|      2|         4|2019-07-20 00:00:00| open_session|
|      2|         4|2019-07-21 00:00:00| send_message|
|      2|         4|2019-07-21 00:00:00|  end_session|
|      3|         2|2019-07-21 00:00:00| open_session|
|      3|         2|2019-07-21 00:00:00| send_message|
|      3|         2|2019-07-21 00:00:00|  end_session|
|      4|         3|2019-06-25 00:00:00| open_session|
|      4|         3|2019-06-25 00:00:00|  end_session|
+-------+----------+-------------------+-------------+



In [31]:
from pyspark.sql.functions import col, count

activity \
.withColumn("day", col("activity_date")) \
.where("day > '2019-06-27' and day <= '2019-07-27'") \
.groupby(["day", "user_id"]) \
.agg(count("activity_type").alias("activities")) \
.where("activities >= 1") \
.groupby("day") \
.agg(count("user_id").alias("active_users")) \
.select(["day", "active_users"]) \
.show()

+-------------------+------------+
|                day|active_users|
+-------------------+------------+
|2019-07-21 00:00:00|           2|
|2019-07-20 00:00:00|           2|
+-------------------+------------+

